In [1]:
import cimgraph.data_profile.cimhub_2023 as cim
from uuid import UUID
from cimgraph.databases import ConnectionParameters
from cimgraph.databases import BlazegraphConnection
from cimgraph.models import FeederModel
from cimgraph.models import DistributedArea
import cimgraph.queries.sparql as sparql
import cimgraph.utils as utils
from defusedxml.ElementTree import parse
cim_profile = 'cimhub_2023'

In [2]:
from cimgraph.databases.fileparsers import XMLFile

params = ConnectionParameters(filename="/home/ande188/CIM-Graph/tests/test_models/ieee13.xml", cim_profile=cim_profile, iec61970_301=8)

xml_file = XMLFile(params)

In [3]:
feeder_mrid = "49AD8E07-3BF9-A4E2-CB8F-C3722F837B62"
# feeder_mrid = "_C1C3E687-6FFD-C753-582B-632A27E28507" #123 node
feeder = cim.Feeder(mRID = feeder_mrid)
network = FeederModel(connection=xml_file, container=feeder, distributed=False)

In [8]:
network.pprint(cim.ACLineSegment)

{
    "a04cdfb1-e951-4fc4-8882-0323cd70ae3c": {
        "identifier": "a04cdfb1-e951-4fc4-8882-0323cd70ae3c",
        "mRID": "A04CDFB1-E951-4FC4-8882-0323CD70AE3C",
        "name": "650632",
        "Location": "3A63B621-3FA6-4805-96F9-DD57A7F88ACF",
        "EquipmentContainer": "49AD8E07-3BF9-A4E2-CB8F-C3722F837B62",
        "BaseVoltage": "2A158E0C-CD01-4A50-AEBA-59D761FCF15D",
        "Terminals": "['000BA1F2-6879-47D7-826D-3CACD23EE20B', '3589A9BE-F3BE-45C3-B84E-3CE5B6105D11']",
        "length": "609.6",
        "ACLineSegmentPhases": "['2A5A1025-AD81-4BFA-A7F0-ED70F7F43DAA', '2D60B241-DE87-48BD-A8D3-FE4B705B40ED', 'A66508A9-C9C1-4DB5-8FFD-F081E5E9E14D']",
        "PerLengthImpedance": "50044320-7295-4591-B39E-F793F1923098"
    },
    "abf53597-a808-422a-b7ee-552f24d83a5f": {
        "identifier": "abf53597-a808-422a-b7ee-552f24d83a5f",
        "mRID": "ABF53597-A808-422A-B7EE-552F24D83A5F",
        "name": "632670",
        "Location": "216C1943-900F-4939-92FE-A3A4817363D8",
  

In [4]:
graph = {}
params = ConnectionParameters(url = "http://localhost:8889/bigdata/namespace/kb/sparql", cim_profile=cim_profile, iec61970_301=7)
database = BlazegraphConnection(params)

In [5]:
rdf = '''{http://www.w3.org/1999/02/22-rdf-syntax-ns#}'''
# Parse the XML file
tree = parse('./test_models/ieee13_assets.xml')
# tree = parse('./test_models/ieee9500bal.xml')
root = tree.getroot()

class_index = {}

# # Function to extract tag without namespace
# def extract_tag(tag):
#     return tag.split('}')[-1]

# Iterate over the elements and create dataclass instances
for element in root:

    class_name = element.tag.split('{'+params.namespace+'}')[1]
    cim_class = eval(f'cim.{class_name}')

    if class_name in cim.__all__:
        uri = element.get(f'{rdf}about')
        identifier = UUID(uri.strip('_').lower())
        uri = uri.split(':')[-1]  # Extract UUID from the full URI
        class_index[identifier] = cim_class
        obj = database.create_object(graph, cim_class, uri)

for element in root:      
    class_name = element.tag.split('{'+params.namespace+'}')[1]
    cim_class = eval(f'cim.{class_name}')

    if class_name in cim.__all__:
        uri = element.get(f'{rdf}about')
        
        identifier = UUID(uri.strip('_').lower())
        uri = uri.split(':')[-1]  # Extract UUID from the full URI
        
        obj = graph[cim_class][UUID(uri)]
        for sub_element in element:
            sub_tag = sub_element.tag.split('}')[-1]
            association = database.check_attribute(cim_class, sub_tag)
            try:
                edge_uri = sub_element.attrib[f'{rdf}resource'].split('uuid:')[-1]
            except:
                edge_uri = None

            if edge_uri is not None:
                if database.namespace not in edge_uri:
                    edge_uuid = UUID(edge_uri.strip('_').lower())                  
                    edge_class = class_index[edge_uuid]
                    database.create_edge(graph, cim_class, identifier, sub_tag, edge_class, edge_uri)
                    reverse = cim_class.__dataclass_fields__[association].metadata['inverse']
                    database.create_edge(graph, edge_class, edge_uuid, reverse, cim_class, graph[cim_class][identifier].uri())
                else:
                    enum_text = edge_uri.split(database.namespace)[1]
                    enum_text = enum_text.split('>')[0]
                    enum_class = enum_text.split('.')[0]
                    enum_value = enum_text.split('.')[1]
                    edge_enum = eval(f'cim.{enum_class}(enum_value)')
                    if association is not None:
                        setattr(graph[cim_class][identifier], association, edge_enum)
            else:
                if association is not None:
                    setattr(obj, association, sub_element.text)


In [6]:
len(graph[cim.ACLineSegment].keys())

11

In [7]:
enum_class

'WireUsageKind'